## ML Pipelines ZenML

##### Install required zenml integrations
``` zenml integration install sklearn -y ```
##### Initialize ZenML
```zenml init```

##### Start the ZenServer to enable dashboard access
```zenml up```

##### Run the feature engineering pipeline
```python run.py --feature-pipeline```

##### Run the training pipeline
```python run.py --training-pipeline```

##### Run the training pipeline with versioned artifacts
```python run.py --training-pipeline --train-dataset-version-name=1 --test-dataset-version-name=1```

##### Run the inference pipeline
```python run.py --inference-pipeline```

### Training the model old fashion way

In [2]:
# import dependendies 
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [3]:
def train_test() -> None:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False)
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    print(f"Test Accuracy: {test_acc}")

train_test()

Test Accuracy: 0.9583333333333334


### Turning experiments into ML pipelines with ZenML

In [ ]:
!zenml init

In [4]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple
from zenml import pipeline

NumExpr defaulting to 8 threads.


In [5]:
# Loading the data
@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """Load the digits dataset as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False)
    return X_train, X_test, y_train, y_test

In [6]:
# Training the model
@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
    """Train a sklearn SVC Classifier"""
    model = SVC(gamma=0.001)
    model.fit(X_train,y_train)
    return model

In [7]:
# Evaluating the model performance
@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
    """Calculate the test set accuracy of an sklearn model."""
    test_acc = model.score(X_test,y_test)
    print(f"Test Accuracy: {test_acc}")
    return test_acc

In [18]:
# Integrating the diferent steps in one pipeline
@pipeline
def digits_pipeline():
    """Links all the steps together in a pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train=X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

In [9]:
# Now running ZenML Pipelines
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered pipeline version: (version: 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of evaluator.
Step evaluator has started.
Pipeline run has finished in 1.696s.


##### Run this command in the terminal 
```zenml up --blocking --port 9004```

In [26]:
!zenml connect --url http://127.0.0.1:9004

NumExpr defaulting to 8 threads.
Connecting to: 'http://127.0.0.1:9004'...
If your browser did not open automatically, please open the following URL into your browser to proceed with the authentication:

http://127.0.0.1:9004/devices/verify?device_id=cb915a47-5793-483a-9622-d83465d4cb73&user_code=451bf3837aa5bb4793a63210a0e9f7f4

Successfully logged in.
Updated the global store configuration.


In [30]:
import webbrowser
webbrowser.open('http://127.0.0.1:9004')

True